In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import idx2numpy

Torch Dataset hazırlama hazırlıkları


In [ ]:
MNIST_DIR = "mnist/"

X_mnist = idx2numpy.convert_from_file(MNIST_DIR + "train-images-idx3-ubyte")
X_mnist = X_mnist.reshape(60000, -1) / 255.0

y_mnist = idx2numpy.convert_from_file(MNIST_DIR + "train-labels-idx1-ubyte")
print(y_mnist[:5])
print(np.unique(y_mnist))

The number 42 is, in The Hitchhiker's Guide to the Galaxy by Douglas Adams, the "Answer to the Ultimate Question of Life, the Universe, and Everything", calculated by an enormous supercomputer named Deep Thought over a period of 7.5 million years. Unfortunately, no one knows what the question is.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_mnist, y_mnist, test_size=0.2, random_state=42)  

x = torch.from_numpy(X_train.astype(np.float32))
y = torch.from_numpy(y_train.astype(np.int64))

print(x.shape)
print(y.shape)

Torch Dataset'ten türeyen kendi classımızı oluşturma

In [ ]:
class MnistDataset(Dataset):
    def __init__(self, X, y):
        self.x = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.int64))

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.x.shape[0]

In [ ]:
dataset = MnistDataset(X_train, y_train)

first_x, first_y = dataset[42]

plt.figure(figsize=(3, 3))
plt.title(first_y.item())
plt.imshow(first_x.reshape(28, 28), cmap="gray")
plt.show()

Kendi MnistDataset sınıfımızı kullanan DataLoader nesnesi oluşturma

In [ ]:
data_loader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)

data_iterator = iter(data_loader)
x, y = next(data_iterator)
print(f"x.shape: {x.shape}, y.shape:{y.shape}")

In [ ]:
plt.figure(figsize=(3, 3))
plt.title(y[3].item())
plt.imshow(x[3].reshape(28, 28), cmap="gray")
plt.show()

Multi-class classification modeli oluşturma 

Model Mimarisi:
1. Giriş katmanı: 784 -> 32
2. Sonuçları sigmoid ile dönüştürme
3. Çıkış katmanı: 32 -> 10

In [ ]:
class MNistClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(784, 32)
        self.activation = nn.Sigmoid()
        self.output_layer = nn.Linear(32, 10)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.activation(x)
        x = self.output_layer(x)
        return x

Model Eğitimi

In [ ]:
data_loader = DataLoader(dataset=dataset, batch_size=256, shuffle=True)
model = MNistClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01)

losses = []

for epoch in range(40):
    for i, (X, y) in enumerate(data_loader):
        y_hat = model(X)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.item())

        if (i+1) % 10 == 0:
            print(f"epoch: {epoch + 1}, batch: {i + 1}, train loss: {loss.item()}")

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data.size())

Hata Görselleştirme

In [ ]:
plt.figure(figsize=(12, 4))
plt.title("Loss")
plt.plot(losses)
plt.xlabel("Time")
plt.ylabel("BCE Loss")
plt.show()

In [ ]:
first_n = 200
plt.figure(figsize=(12, 6))
plt.title(f"First {first_n} Loss Values")
plt.plot(losses[:first_n])
plt.xlabel("Time")
plt.ylabel("BCE Loss")
plt.show()

Test Seti ile Modelin Başarısını Hesaplama

In [ ]:
with torch.no_grad():
    y_predictions = model(torch.from_numpy(X_test.astype(np.float32)))
print(y_predictions[:3])

Logitlerden ağırlıklarından sınıflara...

In [ ]:
y_predicted_classes = torch.argmax(y_predictions, dim=1)

In [ ]:
y_predicted_classes[:3]

In [ ]:
y_test[:3]

In [ ]:
acc = accuracy_score(y_test, y_predicted_classes)
print(acc)